# Simple RAG (Retrieval-Augmented Generation) Model

This notebook demonstrates a basic RAG system that retrieves relevant documents and uses them to generate answers.

In [7]:
# import subprocess
# import sys

# # Install required packages
# subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "langchain", "openai", "faiss-cpu", "sentence-transformers"])
# print("✓ Packages installed successfully!")

In [8]:
# If you encounter import errors, this ensures required packages are installed in the notebook environment.
%pip install -q langchain openai faiss-cpu sentence-transformers langchain-openai


Note: you may need to restart the kernel to use updated packages.


'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: c:\Python312\python.exe -m pip install --upgrade pip


In [9]:

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Step 1: Create sample documents
documents = """
Python is a high-level programming language known for its simplicity and readability.
It was created by Guido van Rossum in 1991.

Machine Learning is a subset of artificial intelligence that enables systems to learn and improve from experience.
Common ML algorithms include decision trees, neural networks, and support vector machines.

Natural Language Processing (NLP) is a field of AI that focuses on interactions between computers and human language.
NLP applications include chatbots, sentiment analysis, and machine translation.

Data Science combines statistics, programming, and domain knowledge to extract insights from data.
Data scientists use tools like Python, R, SQL, and Tableau for their work.
"""

# Step 2: Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_text(documents)
print(f"Created {len(chunks)} document chunks")

# Step 3: Create embeddings using HuggingFace (free, no API key needed)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("✓ Embeddings model loaded")

# Step 4: Create FAISS vector store
vector_store = FAISS.from_texts(chunks, embeddings)
print("✓ Vector store created")

# Step 5: Set up the RAG chain with OpenAI
# Use langchain.chat_models.ChatOpenAI and provide the API key via environment variable for safety.
# Set your key in the notebook or environment: %env OPENAI_API_KEY=sk-...
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

print("✓ RAG chain ready!")

Created a chunk of size 205, which is longer than the specified 200


Created 4 document chunks


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Step 6: Test the RAG system with queries
test_queries = [
    "What is Python?",
    "Tell me about Machine Learning",
    "What is NLP used for?",
    "Who was Python created by?"
]

print("=" * 60)
print("RAG MODEL RESULTS")
print("=" * 60)

for query in test_queries:
    print(f"\n🔍 Query: {query}")
    print("-" * 60)
    
    result = rag_chain.invoke({"query": query})
    
    print(f"Answer: {result['result']}")
    print(f"\nSource Documents:")
    for i, doc in enumerate(result.get('source_documents', []), 1):
        print(f"  [{i}] {doc.page_content[:100]}...")
    print("-" * 60)

## Ask Your Own Question

Edit the cell below and replace the question with anything you want to ask about Python, Machine Learning, NLP, or Data Science!

In [ ]:
# Ask your own question
custom_query = "What are common Machine Learning algorithms?"

print(f"\n🎯 Your Question: {custom_query}")
print("=" * 60)

result = rag_chain.invoke({"query": custom_query})

print(f"\n✨ Answer:\n{result['result']}")
print(f"\n📚 Retrieved from:")
for i, doc in enumerate(result.get('source_documents', []), 1):
    print(f"  [{i}] {doc.page_content}")